In [1]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
import pandas as pd
from simanneal import Annealer

In [5]:
# Load dataset
data = pd.read_csv("dataset_1.csv")  # Replace with your file path

In [7]:
# Feature selection
X = data[["Depth", "WOB", "SURF_RPM", "PHIF", "VSH"]]  # Input features
y = data["ROP_AVG"]  # Target feature

In [9]:
# Normalize/Standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [11]:
# Train an XGBoost surrogate model
surrogate_model = XGBRegressor(n_estimators=100, random_state=42)
surrogate_model.fit(X_scaled, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [13]:
# Define the objective function for Simulated Annealing
def objective_function(x):
    # x is a list of parameters: [Depth, WOB, SURF_RPM, PHIF, VSH]
    # Scale the input
    X_new = scaler.transform([x])
    # Predict ROP_AVG using the XGBoost model
    rop = surrogate_model.predict(X_new)
    return -rop[0]  # Negative sign because SA minimizes the objective function

In [15]:
# Define parameter bounds
param_bounds = {
    "Depth": (data["Depth"].min(), data["Depth"].max()),
    "WOB": (data["WOB"].min(), data["WOB"].max()),
    "SURF_RPM": (data["SURF_RPM"].min(), data["SURF_RPM"].max()),
    "PHIF": (data["PHIF"].min(), data["PHIF"].max()),
    "VSH": (data["VSH"].min(), data["VSH"].max())
}

In [17]:
# Define the Simulated Annealing problem
class ROPOptimizer(Annealer):
    def __init__(self, initial_state):
        super(ROPOptimizer, self).__init__(initial_state)
    
    def move(self):
        # Randomly perturb one of the parameters
        idx = np.random.randint(0, len(self.state))
        self.state[idx] += np.random.uniform(-0.1, 0.1) * (param_bounds[list(param_bounds.keys())[idx]][1] - param_bounds[list(param_bounds.keys())[idx]][0])
        # Ensure the parameter stays within bounds
        self.state[idx] = np.clip(self.state[idx], param_bounds[list(param_bounds.keys())[idx]][0], param_bounds[list(param_bounds.keys())[idx]][1])
    
    def energy(self):
        return objective_function(self.state)

In [19]:
# Initialize the state with random values within bounds
initial_state = [
    np.random.uniform(param_bounds["Depth"][0], param_bounds["Depth"][1]),
    np.random.uniform(param_bounds["WOB"][0], param_bounds["WOB"][1]),
    np.random.uniform(param_bounds["SURF_RPM"][0], param_bounds["SURF_RPM"][1]),
    np.random.uniform(param_bounds["PHIF"][0], param_bounds["PHIF"][1]),
    np.random.uniform(param_bounds["VSH"][0], param_bounds["VSH"][1])
]

In [21]:
# Create the optimizer
optimizer = ROPOptimizer(initial_state)

In [ ]:
# Run Simulated Annealing
optimizer.set_schedule(optimizer.auto(minutes=0.2))  # Run for 0.2 minutes (adjust as needed)
best_state, best_energy = optimizer.anneal()

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
 Temperature        Energy    Accept   Improve     Elapsed   Remaining
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-p

In [ ]:
# Convert the best ROP back to positive (since we minimized the negative)
best_rop = -best_energy

In [ ]:
# Print results
print("Best Parameters (Depth, WOB, SURF_RPM, PHIF, VSH):", best_state)
print("Optimized ROP_AVG:", best_rop)